In [31]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt

%matplotlib inline

our_data = 'poop_data/Denoising'
data_dir = 'poop_data'

In [30]:
pwd

'/home/jovyan/Poop-Power'

With the follwoing step we insert our sequences and we analyzed their data format. It appears that are paired end sequences and this is an important information that allows us to select a certain typ of denoising command.

In [33]:
! qiime tools peek $data_dir/sequences.qza

UUID:        32a1795b-d6fb-4ecc-9166-4fe29fb8206a
Type:        SampleData[PairedEndSequencesWithQuality]
Data format: SingleLanePerSamplePairedEndFastqDirFmt


In [34]:
! qiime demux summarize \
    --i-data $data_dir/sequences.qza \
    --o-visualization $data_dir/sequences.qzv

Saved Visualization to: poop_data/sequences.qzv


In [35]:
Visualization.load(f'{data_dir}/sequences.qzv')

<visualization: Visualization uuid: be4423bc-9770-4d07-89e8-263a3c505fb2>

### Denoising - Amplicon Sequence Variants (ASV)
At first we want to denoise the data to create a feature table of ASVs. Our stept to apply Denoising (= remove noise) are the following):
1.Find the cleanest sequence (and/or model probabilities based on quality scores)
2.Correct and/or discard super noisy sequences (high expected errors)
3.DADA2: DADA2 builds an error model which can identify differences between sequences, filters out noisy sequences and generates a feature table with error-corrected sequences.


In [36]:
! qiime dada2 denoise-paired \
    --i-demultiplexed-seqs $data_dir/sequences.qza \
    --p-trunc-len-f 131 \
    --p-trunc-len-r 145 \
    --p-n-threads 3 \
    --o-table $our_data/dada2_table.qza \
    --o-representative-sequences $our_data/dada2_rep_set.qza \
    --o-denoising-stats $our_data/dada2_stats.qza

Saved FeatureTable[Frequency] to: poop_data/Denoising/dada2_table.qza
Saved FeatureData[Sequence] to: poop_data/Denoising/dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: poop_data/Denoising/dada2_stats.qza


In [37]:
! qiime metadata tabulate \
    --m-input-file $our_data/dada2_stats.qza \
    --o-visualization $our_data/dada2_stats.qzv

Saved Visualization to: poop_data/Denoising/dada2_stats.qzv


In [38]:
Visualization.load(f'{our_data}/dada2_stats.qzv')

<visualization: Visualization uuid: 59f690bd-7d89-487d-8cb6-f19046c3eeab>

In [39]:
! qiime feature-table tabulate-seqs \
    --i-data $our_data/dada2_rep_set.qza \
    --o-visualization $our_data/dada2_rep_set.qzv

Saved Visualization to: poop_data/Denoising/dada2_rep_set.qzv


In [40]:
Visualization.load(f'{our_data}/dada2_rep_set.qzv')

<visualization: Visualization uuid: a59fbeb4-35ae-4239-8e52-d25f35b8937b>

### Clustering - Operational Taxonomic Units (OUT)
With Clustering we are going to remove noisy sequences and reduce the amount of sequences to process. It works based on a given threshold, i.e. 97% similarity. Is less accurate then the denoising method.-->not needed